In [1]:
from pathlib import Path
import datetime
import numpy as np
import pandas as pd

In [2]:
benchmark = "construction-set"
benchmarkfolder = Path(".").joinpath(benchmark)
file = benchmarkfolder.joinpath("benchmark")
file

PosixPath('construction-set/benchmark')

In [3]:
t = datetime.datetime.strptime("0:00:10.917553", "%H:%M:%S.%f").time()
t

datetime.time(0, 0, 10, 917553)

In [4]:
delta = datetime.timedelta(hours=t.hour, minutes=t.minute, seconds=t.second, microseconds=t.microsecond)
delta

datetime.timedelta(seconds=10, microseconds=917553)

In [5]:
delta.total_seconds()

10.917553

In [6]:
def parse_time(line: str):
    try:
        t = datetime.datetime.strptime(line, "%H:%M:%S.%f").time()
    except ValueError:
        return None
    return datetime.timedelta(
        hours=t.hour, minutes=t.minute, seconds=t.second, microseconds=t.microsecond
    ).total_seconds()

In [7]:
parse_time("0:00:10.917553")

10.917553

In [8]:
k, v = "Number nodes: 208".strip().split(':')
print(k)
print(int(v.strip()))

Number nodes
208


In [12]:
def parse_numnode(line:str):
    return int(line.strip().split(':', 1)[1].strip())

In [13]:
parse_numnode("Number nodes: 208")

208

In [22]:
def parse_file(file):
    timelist = []
    numnodelist = []
    with file.open("r") as f:
        for line in f.readlines():
            if "#" in line:
                break
            if "Failed to plan path" in line:
                continue
            line = line.strip()
            t = parse_time(line)
            if not t:
                numnodelist.append(parse_numnode(line))
            else:
                timelist.append(t)
    return timelist, numnodelist

In [23]:
timelist, numnodelist = parse_file(file)

In [25]:
print(np.average(timelist))
print(np.average(numnodelist))

51.10097526
1024.38


In [26]:
csvfile = benchmarkfolder.joinpath("benchmark.csv")
csvfile

PosixPath('construction-set/benchmark.csv')

In [27]:
listdict = {'Solving time (s)': timelist, 'Number of nodes': numnodelist}
df = pd.DataFrame(listdict) 
df.to_csv(csvfile, index=False)

In [28]:
benchmarkfolder

PosixPath('construction-set')

In [29]:
currentfolder = Path(".")
currentfolder

PosixPath('.')

In [30]:
benchmarkfolderlist = [f for f in currentfolder.iterdir() if f.is_dir() and ".ipynb" not in f.stem]
benchmarkfolderlist

[PosixPath('baxter-manipulation-boxes'),
 PosixPath('pr2-manipulation-kitchen-spf'),
 PosixPath('romeo-placard'),
 PosixPath('ur3-spheres-spf'),
 PosixPath('baxter-manipulation-boxes-spf'),
 PosixPath('pr2-in-iai-kitchen'),
 PosixPath('pyrene-on-the-ground'),
 PosixPath('pr2-manipulation-two-hand'),
 PosixPath('ur3-spheres'),
 PosixPath('pr2-manipulation-two-hand-spf'),
 PosixPath('romeo-placard-spf'),
 PosixPath('construction-set-spf'),
 PosixPath('pr2-manipulation-kitchen'),
 PosixPath('construction-set')]

In [31]:
[f for f in benchmarkfolder.iterdir() if f.is_file() and not f.suffix]

[PosixPath('construction-set/benchmark')]

In [32]:
f = next(benchmarkfolder.iterdir())
f.stem

'benchmark'

In [33]:
currentfolder = Path(".")
benchmarkfolderlist = [f for f in currentfolder.iterdir() if f.is_dir() and ".ipynb" not in str(f)]
for benchmarkfolder in benchmarkfolderlist:
    filelist = [f for f in benchmarkfolder.iterdir() if f.is_file() and not f.suffix]
    for file in filelist:
        timelist, numnodelist = parse_file(file)
        listdict = {'Solving time (s)': timelist, 'Number of nodes': numnodelist}
        csvfile = benchmarkfolder.joinpath(file.name).with_suffix(".csv")
        df = pd.DataFrame(listdict) 
        df.to_csv(csvfile, index=False)